In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.impute import KNNImputer

In [2]:
df = pd.read_csv("qm9.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133885 entries, 0 to 133884
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   mol_id  133885 non-null  object 
 1   smiles  133885 non-null  object 
 2   A       133885 non-null  float64
 3   B       133885 non-null  float64
 4   C       133885 non-null  float64
 5   mu      133885 non-null  float64
 6   alpha   133885 non-null  float64
 7   homo    133885 non-null  float64
 8   lumo    133885 non-null  float64
 9   gap     133885 non-null  float64
 10  r2      133885 non-null  float64
 11  zpve    133885 non-null  float64
 12  u0      133885 non-null  float64
 13  u298    133885 non-null  float64
 14  h298    133885 non-null  float64
 15  g298    133885 non-null  float64
 16  cv      133885 non-null  float64
dtypes: float64(15), object(2)
memory usage: 17.4+ MB


In [4]:
num_rows, num_columns = df.shape
print("Количество строк:", num_rows)
print("Количество столбцов:", num_columns)
#типы данных в каждом столбце
print(df.dtypes)

Количество строк: 133885
Количество столбцов: 17
mol_id     object
smiles     object
A         float64
B         float64
C         float64
mu        float64
alpha     float64
homo      float64
lumo      float64
gap       float64
r2        float64
zpve      float64
u0        float64
u298      float64
h298      float64
g298      float64
cv        float64
dtype: object


In [5]:
random_state = 38

In [6]:
sampled_data = df.sample(n=20000, random_state=random_state)

In [7]:
print(sampled_data.head())

            mol_id            smiles        A        B        C      mu  \
109094  gdb_109095     CCC1=CC=CCCC1  3.03486  1.07556  0.86799  0.6913   
126738  gdb_126739     CCn1ccc(n1)OC  3.12802  1.11330  0.87300  1.9682   
61920    gdb_61921    CC(NCC#N)C1CN1  3.23759  0.78411  0.66376  4.9958   
17598    gdb_17599  CC(=O)C(=O)N1CC1  3.92086  1.63625  1.19988  1.3069   
94339    gdb_94340    OC1CCC11CC1C#C  2.61019  1.28615  1.07855  2.0761   

        alpha    homo    lumo     gap         r2      zpve          u0  \
109094  97.97 -0.2028 -0.0147  0.1880  1437.9297  0.207597 -351.182105   
126738  79.80 -0.2077  0.0319  0.2396  1364.4040  0.159883 -419.227054   
61920   80.82 -0.2486  0.0209  0.2695  1669.6409  0.170769 -399.297998   
17598   64.00 -0.2403 -0.0671  0.1732   978.7413  0.116682 -399.810511   
94339   82.73 -0.2360  0.0475  0.2835  1195.5554  0.157575 -385.829753   

              u298        h298        g298      cv  
109094 -351.172934 -351.171990 -351.216386  35.192 

только определённые строки из исходного набора

In [8]:
df = sampled_data

In [9]:
print(df.head())

            mol_id            smiles        A        B        C      mu  \
109094  gdb_109095     CCC1=CC=CCCC1  3.03486  1.07556  0.86799  0.6913   
126738  gdb_126739     CCn1ccc(n1)OC  3.12802  1.11330  0.87300  1.9682   
61920    gdb_61921    CC(NCC#N)C1CN1  3.23759  0.78411  0.66376  4.9958   
17598    gdb_17599  CC(=O)C(=O)N1CC1  3.92086  1.63625  1.19988  1.3069   
94339    gdb_94340    OC1CCC11CC1C#C  2.61019  1.28615  1.07855  2.0761   

        alpha    homo    lumo     gap         r2      zpve          u0  \
109094  97.97 -0.2028 -0.0147  0.1880  1437.9297  0.207597 -351.182105   
126738  79.80 -0.2077  0.0319  0.2396  1364.4040  0.159883 -419.227054   
61920   80.82 -0.2486  0.0209  0.2695  1669.6409  0.170769 -399.297998   
17598   64.00 -0.2403 -0.0671  0.1732   978.7413  0.116682 -399.810511   
94339   82.73 -0.2360  0.0475  0.2835  1195.5554  0.157575 -385.829753   

              u298        h298        g298      cv  
109094 -351.172934 -351.171990 -351.216386  35.192 

In [10]:
df

,mol_id,smiles,A,B,C,mu,alpha,homo,lumo,gap,r2,zpve,u0,u298,h298,g298,cv
109094,gdb_109095,CCC1=CC=CCCC1,3.03486,1.07556,0.86799,0.6913,97.97,-0.2028,-0.0147,0.1880,1437.9297,0.207597,-351.182105,-351.172934,-351.171990,-351.216386,35.192
126738,gdb_126739,CCn1ccc(n1)OC,3.12802,1.11330,0.87300,1.9682,79.80,-0.2077,0.0319,0.2396,1364.4040,0.159883,-419.227054,-419.217941,-419.216997,-419.261636,32.830
61920,gdb_61921,CC(NCC#N)C1CN1,3.23759,0.78411,0.66376,4.9958,80.82,-0.2486,0.0209,0.2695,1669.6409,0.170769,-399.297998,-399.288094,-399.287150,-399.333993,35.251
17598,gdb_17599,CC(=O)C(=O)N1CC1,3.92086,1.63625,1.19988,1.3069,64.00,-0.2403,-0.0671,0.1732,978.7413,0.116682,-399.810511,-399.802101,-399.801156,-399.844284,28.769
94339,gdb_94340,OC1CCC11CC1C#C,2.61019,1.28615,1.07855,2.0761,82.73,-0.2360,0.0475,0.2835,1195.5554,0.157575,-385.829753,-385.820728,-385.819784,-385.863532,34.636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50347,gdb_50348,O=CC1OC2CCC1O2,2.96810,1.58096,1.50542,2.2867,66.90,-0.2542,-0.0346,0.2196,940.7575,0.137406,-458.974668,-458.967525,-458.966581,-459.006630,27.430
108075,gdb_108076,OCC12OC3CC1(O)C23,2.70275,1.70254,1.31010,0.9403,67.04,-0.2260,0.0509,0.2769,987.0831,0.135449,-458.882355,-458.874610,-458.873666,-458.914492,30.611
29604,gdb_29605,Cc1cocc1NC=O,2.96825,1.29025,0.90440,4.2128,73.70,-0.2117,0.0026,0.2142,1225.3559,0.124686,-437.941730,-437.933372,-437.932428,-437.975111,30.422
86261,gdb_86262,CC1CC(C)C1(C)C#N,1.71625,1.62544,1.12788,4.1443,86.51,-0.2923,0.0344,0.3268,1240.4082,0.193300,-367.235662,-367.225728,-367.224784,-367.269779,37.636


In [11]:
df.to_csv('qm9.csv', index=False)
df = pd.read_csv("qm9.csv")
df.to_csv('qm9_1.csv', index=False)
df.to_csv('qm9_1.csv', index=False)